# Titanic - Logistic Regression Baseline

## Trainning model

In [1]:
import os
import sys
import joblib
import pandas as pd
import numpy as np
from os.path import join
from sklearn.linear_model import LogisticRegression
from tuningModel import tune_logistic_regression
# === Load dữ liệu ===
path_dir = join("..", "..", "..")
input_dir = join(path_dir, "data", "feature_engineered", "scaledData")

df_train = pd.read_csv(join(input_dir, "scaledData_engineered_train.csv"))
df_test = pd.read_csv(join(input_dir, "scaledData_engineered_test.csv"))

X = df_train.drop(['Survived'], axis=1)
y = df_train['Survived']
# === Chuẩn hóa dữ liệu (giúp Logistic Regression hội tụ ổn định hơn) ===
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test_scaled = scaler.transform(df_test.drop(columns=['Survived'], errors='ignore'))

best_model, result_df = tune_logistic_regression(X,y)
# === KFold ===
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies, precisions, recalls, f1s = [], [], [], []

fold_index = 1
for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # === Mô hình Logistic Regression ===
    model = best_model
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    # === Metrics ===
    acc = accuracy_score(y_val, y_pred)
    prec = precision_score(y_val, y_pred)
    rec = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1s.append(f1)

    print(f"Fold {fold_index} - Acc: {acc:.4f} | F1: {f1:.4f}")
    fold_index += 1

# === Mean & Std ===
mean_acc = np.mean(accuracies)
mean_prec = np.mean(precisions)
mean_rec = np.mean(recalls)
mean_f1 = np.mean(f1s)
std_acc = np.std(accuracies)

print("\n==== Mean metrics ====")
print(f"Accuracy: {mean_acc:.4f}")
print(f"Precision: {mean_prec:.4f}")
print(f"Recall: {mean_rec:.4f}")
print(f"F1-score: {mean_f1:.4f}")
print(f"Std (Accuracy): {std_acc:.4f}")



Fitting 5 folds for each of 60 candidates, totalling 300 fits

 Best Score (accuracy): 0.7991
 Best Params: {'C': 0.01, 'l1_ratio': 0, 'penalty': 'l2', 'solver': 'saga'}
Fold 1 - Acc: 0.8156 | F1: 0.7556
Fold 2 - Acc: 0.7472 | F1: 0.6087
Fold 3 - Acc: 0.8427 | F1: 0.7778
Fold 4 - Acc: 0.7640 | F1: 0.6500
Fold 5 - Acc: 0.8202 | F1: 0.7333

==== Mean metrics ====
Accuracy: 0.7980
Precision: 0.7925
Recall: 0.6359
F1-score: 0.7051
Std (Accuracy): 0.0362


c:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning: 
75 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
23 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "c:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py", line 436, in _validate_params

## Save model into logger and dump model to pickle file

In [2]:
# === Import log_experiment ===
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))
from log.experiment_logger import log_experiment

# === Ghi log kết quả vào CSV ===
log_path = join(path_dir, "log", "experiment_log.csv")
log_experiment(
    output_path=log_path,
    model_name="LogisticRegression",
    feature_name="scaledData_engineered",
    params= best_model.get_params(),
    kfold=5,
    f1=mean_f1,
    acc=mean_acc,
    rec=mean_rec,
    prec=mean_prec,
    std=std_acc,
    author="Thang"
)

# === Huấn luyện lại trên toàn bộ dữ liệu train ===
final_model = best_model
final_model.fit(X, y)

# === Lưu model ===
model_dir = join(path_dir, "notebook", "model", "logistic_regression", "Model Pickles")
os.makedirs(model_dir, exist_ok=True)
model_path = join(model_dir, "lr_scaledData_tuned.pkl")
joblib.dump((final_model, scaler), model_path)  # lưu luôn scaler để predict sau
print(f"✅ Model saved to {model_path}")
df_original = pd.read_csv(join(path_dir,"data","raw","test.csv"))
passenger_ids = df_original["PassengerId"]
# === Tạo file submission ===
y_test_pred = final_model.predict(X_test_scaled)

submission = pd.DataFrame({
    'PassengerId': passenger_ids,  # phải có trong test
    'Survived': y_test_pred
})

sub_dir = join(path_dir, "notebook", "model", "logistic_regression", "submissions")
os.makedirs(sub_dir, exist_ok=True)
submission_path = join(sub_dir, "submission_lr_scaledData_tuned.csv")
submission.to_csv(submission_path, index=False)
print(f"📤 Submission file saved to {submission_path}")


Logged experiment to ..\..\..\log\experiment_log.csv
✅ Model saved to ..\..\..\notebook\model\logistic_regression\Model Pickles\lr_scaledData_tuned.pkl
📤 Submission file saved to ..\..\..\notebook\model\logistic_regression\submissions\submission_lr_scaledData_tuned.csv


c:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(


# The end